In [1]:
from preprocessing import transform_data_for_rnn
from data_scraper import scrape_udp_data
import torch
import torch.nn as nn
import numpy as np

In [2]:
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [3]:
class StateEstimationDataset(torch.utils.data.Dataset):
    def __init__(self, motion_dir, telemetry_dir, udp_filename, buffer_size=5):
        scrape_udp_data(motion_dir, telemetry_dir, filename=udp_filename)
        self.X_data, self.y_data = transform_data_for_rnn(udp_filename, buffer_size)
        self.X_data = torch.from_numpy(self.X_data)
        self.y_data = torch.from_numpy(self.y_data)
    def __len__(self):
        return(self.X_data.shape[0])
    def __getitem__(self, idx):
        x = self.X_data[idx]
        y = self.y_data[idx]
        return x, y

udp_dataset = StateEstimationDataset(motion_dir=r"C:\\Users\\chros\\Documents\\Deep Racing\\australia_run1\\udp_data\\motion_packets", telemetry_dir=r"C:\\Users\\chros\\Documents\\Deep Racing\\australia_run1\\udp_data\\car_telemetry packets", udp_filename="udp_data.pkl", buffer_size=5)
split_id = int(len(udp_dataset)* 0.8)
train_dataset, val_dataset = torch.utils.data.random_split(udp_dataset, [split_id, (len(udp_dataset) - split_id)])

Pickle file already exists


In [4]:
batch_size=32
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_data_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [5]:
class StateEstimationNet(nn.Module):
    def __init__(self, buffer_size, input_shape, hidden_dim, n_layers, batch_size, drop_prob=0.5):
        super().__init__()
        self.output_size = 3 * buffer_size
        self.input_shape = input_shape
        self.n_layers = n_layers
        self.batch_size = batch_size
        self.buffer_size = buffer_size
        self.hidden_dim = hidden_dim

        self.lstm = nn.LSTM(input_shape, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, self.output_size)

    def forward(self, x, hidden):
        x = x.float()
        x = torch.unsqueeze(x, 1)
        lstm_out, hidden = self.lstm(x, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        out = self.dropout(lstm_out)
        out = self.fc(out)
        out = out.view(self.batch_size, -1)
        return out, hidden
    
    def init_hidden(self):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, self.batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, self.batch_size, self.hidden_dim).zero_().to(device))
        return hidden


In [6]:
model = StateEstimationNet(buffer_size=5, input_shape=udp_dataset.X_data.shape[-1], hidden_dim=100, n_layers=2, batch_size=batch_size)
model.to(device)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [7]:
print(model)

StateEstimationNet(
  (lstm): LSTM(75, 100, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=100, out_features=15, bias=True)
)


In [8]:
epochs = 100
counter = 0
valid_loss_min = np.Inf

model.train()
for i in range(epochs):
    h = model.init_hidden()
    for inputs, labels in train_data_loader:
        counter += 1
        h = tuple([e.data for e in h])
        inputs, labels = inputs.to(device), labels.to(device)
        model.zero_grad()
        output, h = model(inputs, h)
        loss = loss_function(output.squeeze(), labels.float())
        loss.backward()
        optimizer.step()

        if counter%1000 == 0:
            val_h = model.init_hidden()
            val_losses = []
            model.eval()
            for inp, lab in val_data_loader:
                val_h = tuple([each.data for each in val_h])
                inp, lab = inp.to(device), lab.to(device)
                out, val_h = model(inp, val_h)
                val_loss = loss_function(out.squeeze(), lab.float())
                val_losses.append(val_loss.item())
            model.train()
            if np.mean(val_losses) <= valid_loss_min:
                torch.save(model.state_dict(), './state_dict.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)
    print("Epoch: {}/{}...".format(i+1, epochs),
          "Loss: {:.6f}...".format(loss.item()),
          "Val Loss: {:.6f}".format(np.mean(val_losses)))

oss: 31992.130859... Val Loss: 29070.511518
Epoch: 45/100... Step: 196000... Loss: 66899.242188... Val Loss: 26346.742831
Epoch: 46/100... Step: 197000... Loss: 57689.148438... Val Loss: 30229.158145
Epoch: 46/100... Step: 198000... Loss: 56776.218750... Val Loss: 28543.165963
Epoch: 46/100... Step: 199000... Loss: 63354.886719... Val Loss: 27445.143461
Epoch: 46/100... Step: 200000... Loss: 41640.945312... Val Loss: 28657.009247
Epoch: 47/100... Step: 201000... Loss: 46379.796875... Val Loss: 34117.882486
Epoch: 47/100... Step: 202000... Loss: 96442.929688... Val Loss: 30333.021570
Epoch: 47/100... Step: 203000... Loss: 69316.414062... Val Loss: 29618.582342
Epoch: 47/100... Step: 204000... Loss: 53366.929688... Val Loss: 29160.152422
Epoch: 48/100... Step: 205000... Loss: 66096.453125... Val Loss: 30866.020506
Epoch: 48/100... Step: 206000... Loss: 42556.925781... Val Loss: 26267.502875
Epoch: 48/100... Step: 207000... Loss: 34911.402344... Val Loss: 24280.947449
Epoch: 48/100... Ste